# Chapter 7 — Model Context Protocol (MCP) 
## 1.Overview

**What is MCP?**  
The **Model Context Protocol** is an open, JSON-RPC–based standard that lets AI apps (hosts) connect LLMs to **external tools, data, and environments** via **MCP servers**. Think of it like a **USB-C port for AI**: plug standardized “devices” (tools/resources) into many different apps without bespoke wiring.

---

### Core roles

- **Host**: the user-facing AI app (e.g., an IDE with AI, a chat UI, or your agents app).
- **Client**: the host’s component that manages a **1:1 connection** to an MCP server, handling protocol details.
- **Server**: an external program that **exposes capabilities** (tools, resources, prompts) over MCP.

> One host can connect to many servers at once. One client connects to one server.

---

### Why MCP (the problem it solves)

- Avoids the **M×N integration** tangle (M apps × N tools) by standardizing the interface.  
- Promotes **interoperability** and **modularity**: add/compose new servers without changing hosts.

---

### High-level flow

1. **User input** → host receives intent.  
2. **Host reasoning** → decides which external capabilities might be needed.  
3. **Client connects** → to the relevant server(s).  
4. **Capability discovery** → list tools/resources/prompts.  
5. **Invoke capability** → host (via client) calls a tool on the server.  
6. **Server executes** → returns results.  
7. **Integrate results** → host feeds them into the LLM context or renders to the user.

---

### Transport & messages

- **JSON-RPC 2.0** for message format:
  - **Request** (id, method, params)
  - **Response** (result or error)
  - **Notification** (one-way event)
- **Transports**:
  - **stdio** (local child process; simple, sandboxed by OS)
  - **HTTP + SSE / streamed HTTP** (remote/cloud; works across networks, server-push updates)

---

### Using MCP with the OpenAI Agents SDK (conceptual)

The Agents SDK can consume MCP-exposed tools through an MCP client so your **agent can call them like any other tool**. Two common patterns:

1) **Direct use as tools** — discover MCP tools and register them on your agent.  
2) **Orchestrator agent** — your agent routes among **local function tools**, **agent-as-tool** specialists, and **MCP tools** from external servers in one workflow.

> Exact import paths / helpers can vary by SDK version. The idea: **connect a client, discover tools, adapt them into the agent’s `tools=[...]` list**, and let the model call them under your instructions/guardrails.

---

### How this compares

| Capability type      | Where it runs            | How you define it                  | Best for |
|---|---|---|---|
| **Function tool**    | Your app process         | Python function + `@function_tool` | In-process logic/APIs you own |
| **Agent-as-tool**    | Your app process         | `other_agent.as_tool(...)`         | Composing specialists w/o handoff |
| **MCP tool**         | External MCP server      | MCP server exposes tools           | Reusing external capabilities across apps |

---

### Quick quiz (recap)

- **Main purpose of MCP?** → **Enable LLMs to connect to external data/tools** via a standard (interoperability).  
- **What problem does MCP target?** → The **M×N integration** explosion.  
- **Key advantage?** → **Standardization & interoperability** across the AI ecosystem.  
- **“Host” means…?** → The **user-facing** AI application.  
- **JSON-RPC in MCP?** → It’s the **message format** used between client and server.  
- **MCP SDK’s role?** → Makes it **easier to implement** clients/servers (handles JSON-RPC ser/de, etc.).

---

### Practical tips

- Start with a **local stdio server** during development (easy to run, easy to debug).  
- Keep **instructions** explicit (“Use the MCP tool X when Y happens”) so the model reliably calls it.  
- Use **hooks/streaming** to observe tool discovery/calls and log results in real time.  
- Combine MCP tools with **function tools** and **agent-as-tool** to build rich, modular workflows.

> TL;DR: MCP is a **universal connector** for AI apps. With the Agents SDK, you can treat MCP servers as first-class tool providers—discover capabilities once, reuse everywhere, and keep your orchestration clean.


# 2.MCP: Code Walkthrough (Server + Client)

This section shows a **minimal end-to-end MCP setup**: an MCP **server** that exposes a tiny tool, and an Agents SDK **client** that connects over **SSE** and calls that tool.

In [ ]:
import asyncio
from dotenv import load_dotenv
from agents import Agent, Runner, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
from agents.mcp.server import MCPServerSse
from agents.model_settings import ModelSettings
import os

# --- Env & model ---
load_dotenv()
api_key = os.getenv('API_KEY')

base_url = "https://api.openai.com/v1"  
chat_model = "gpt-4.1-nano-2025-04-14"  
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)


async with MCPServerSse(
    params={
        "url": "xx",
    }
) as my_mcp_server: # 建议用这种上下文写法，否则需要手动连接和关闭MCP服务。
    agent = Agent(
        name="Assistant",
        instructions="你是一个火车票查询助手，能够查询火车票信息。",
        mcp_servers=[my_mcp_server],
        model_settings=ModelSettings(tool_choice="required"),
         model=llm,
    )

    message = "明天从广州到杭州可以买哪些火车票？"
    print(f"Running: {message}")
    result = await Runner.run(starting_agent=agent, input=message)
    print(result.final_output)




Running: 明天从广州到杭州可以买哪些火车票？
以下是明天（2025年10月17日）从广州到杭州的可购火车票信息：
 
1. K512（广州白云 -> 杭州南）：出发07:52，抵达04:24，硬座剩余10张票，价189.5元，无座票也有。
2. K528（广州白云 -> 杭州南）：出发08:18，抵达01:49，硬座有票，价173.5元，无座也有票。
3. G818（广州南 -> 杭州东）：出发08:27，抵达14:16，商务座无票，一等座无票，二等座无票。
4. G1184（广州南 -> 杭州西）：出发08:43，抵达17:28，商务座无票，一等座无票，二等座无票。
5. G3068（广州东 -> 杭州西）：出发09:09，抵达15:58，商务座剩余6张，价2602元，一等座剩余1张，价1257元，无座有票。
6. G3068（广州新塘 -> 杭州西）：出发09:36，抵达15:58，商务座剩余6张，价2520元，一等座剩余1张，价1220元，无座有票。
7. D3121（广州东 -> 杭州东）：出发10:43，抵达22:38，一等座有票，价1031元，无座也有。
8. G3072（广州东 -> 杭州西）：出发11:36，抵达20:24，二等座有票，价758元，其他座位剩余情况不同。
9. G3072（广州新塘 -> 杭州西）：出发12:03，抵达20:24，二等座有票，价727元。
10. G800（广州东 -> 杭州东）：出发12:12，抵达18:36，二等座有票，价820元。
11. 其他车次也有不同座席剩余情况。

请告诉我是否需要帮你预订或查找特定车型的票。
